In [1]:
from vllm import LLM, SamplingParams
import sys

sys.path.append("..")

from helpers import get_conv_template

In [2]:
# Create an LLM.
llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")

/home/sidnbaskaran/miniconda3/envs/editor/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

INFO 06-08 16:23:02 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

INFO 06-08 16:23:33 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-08 16:23:36 gpu_executor.py:83] # GPU blocks: 9623, # CPU blocks: 2048
INFO 06-08 16:23:38 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-08 16:23:38 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-08 16:23:45 model_runner.py:924] Graph capturing finished in 6 secs.


In [16]:
# Sample prompts.
system_prompt = """Consider the sentence below. Identify its main subject entity.
Write a short sentence inventing a new piece of information about that entity, which ought to change the continuation

Example:
Input:
Sentence: Altered Carbon is a 2002 British cyberpunk novel by the English writer Richard K. Morgan.

Output:
<result>
Entity: Altered Carbon
New Context: Altered Carbon was written in 1994
<result/>
"""

prompt = "His Last Haul is a 1928 American silent crime drama film directed by Marshall Neilan and starring Tom Moore, Seena Owen and Alan Roscoe."
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, top_k=50, max_tokens=128)

conv = get_conv_template("llama3")
conv.system_message = system_prompt
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], "")

out = llm.generate(conv.get_prompt(), sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s, Generation Speed: 63.21 toks/s]


In [17]:
print(out[0].outputs[0].text)

Entity: His Last Haul
New Context: His Last Haul was a lost film, and only a few still images of the movie's posters and lobby cards remain.


 - check if the entity is correct
 - subset original sentence based on entity
 - generate new continuation using gpt2 (we can use vllm again for this)
 - careful about tokenization
